In [85]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets ,models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [86]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomDNADataset(Dataset):
    def __init__(self, feature_file,label_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(label_file)
        self.img_features = pd.read_csv(feature_file)
        # pandas read the data 
        self.transform = transform
        self.target_transform = target_transform
        # no transform is used here 
        #self.dic= {'-':0,'A' :2**2 , 'C':2**4,'T':2**6,'G':2**7,'N':0}
        self.dic= {'-':[0,0,0,0],'A' :[1,0,0,0] , 'C':[0,1,0,0],'T':[0,0,1,0],'G':[0,0,0,1]}
        # I learned the sequence of dana from file:///D:/downloads/dna-and-animal-classification.pdf
        #and from http://ircamera.as.arizona.edu/Astr2016/text/nucleicacid1.htm 
        # which showed that there are 4 dominante letters to determine the sequence

    def __len__(self):
        return len(self.img_labels)
        # this is just the number rows or samples in the input data as used by 
        # Dataset class 

    def __getitem__(self, idx):
 ############# DNA transform ######################     
        DNA = self.img_features.iloc[idx,1]
        # so DNA is getting the img_features panda csv read and we are taking the second
        #colmn , the first is just ids . idx is generated by pytorch randomly depending on
        # if we shuffle the data or not and the number of batch size (part of Dataset class which
        # we inheret from )


        n = np.array([])
        Our_pad = (36*36)-len(DNA)
        # here I am using 1296 which is 36*36 , I choose this number based on 
        # the maximum individual row I found which was around 1058 ,this is used to 
        # generate a padding which is some how consistent along all .I also chose 
        # 36 so that when we do conv and maxpool we get a nice number which is 
        # dividable by 2 How to fix RuntimeError "Expected object 
        #of scalar type Float but got scalar type Double for argument"
        # here to get the right dim we muliply by 2 the in dim 
        
        for i in range(len(DNA)):# DNA is our row where we itterate 

          if DNA[i] in self.dic:
            mut = self.dic[DNA[i]]
            mut = [x * 2 for x in mut]

            n = np.append(n,mut)
          else:
            n= np.append(n,[0,0,0,0])
          
          
        for pad in range(Our_pad):# we pad the end of the sequence with zeros 
          n =np.append(n,[0,0,0,0])

        
        l = np.array(n , dtype=np.float32)
        # this was very trick where the base type of any np array is float 64 or 
        # double but the base type for torch is float 32 so if we convert 
        # a np array to torch we need to first change it to float32 or we get an error 
        # which says 
        Data_array = torch.from_numpy(l)
        #transfer a np to torch 
        #DNA_dense = Data_array
        # depending on the type of network we can keep it flat or 
        # change the dim (resh
        DNA_image = Data_array.reshape(1,72,72)

        # reshape the dim to be in the form of an image of pix 1 which is 
        # usally the RGB or gray channel , in our case it is gray ,
        # then 36 by 36 which what our padding is doing 
        #for a 3 chennel image insted of 1 


#################################################
        label = self.img_labels.iloc[idx, 1]
        # we get the labels from a different file 

        return DNA_image, label

In [87]:
train_features_path = 'C:\\Users\\yousi\\Untitled Folder\\dna-barcode-classification/train_features.csv'
test_features_path = 'C:\\Users\\yousi\\Untitled Folder\\dna-barcode-classification/test_features.csv'
train_labels_path = 'C:\\Users\\yousi\\Untitled Folder\\dna-barcode-classification/train_labels.csv'
test_labels_path = 'C:\\Users\\yousi\\Untitled Folder\\dna-barcode-classification/train_labels - Copy.csv'


full_dataset= CustomDNADataset(feature_file=train_features_path,label_file=train_labels_path)
# so pass the whole set by specifing the path of file to be inputed to our custum class 

Test_dataset = CustomDNADataset(feature_file=test_features_path,label_file=test_labels_path)
# for the test_dataset I am using fake label data which is not accually there just so that I can use 
# my custom class , it has the same len as the train data ( I choped some in csv file )

In [88]:
train_size = int(0.9 * len(full_dataset))
# this is a creative way to create a validation set by spliting the dataset 
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])


In [89]:

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 


# hyper parameters 
input_size = 72
sequence_length = 72
num_layers = 2
hidden_size  = 256

num_classes = 1214
learning_rate = 0.001
batch_size = 15
# the batch size appered to be a significant factor on the effectivness of the training
# 15 is found to be a good num 
num_epochs = 3
#


In [90]:
class RNN(nn.Module):
    def __init__(self, input_size , hidden_size , num_layers , num_classes):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size , hidden_size , num_layers,batch_first= True)
        self.fc = nn.Linear(hidden_size*sequence_length ,num_classes)
        
        
    def forward(self,x):
        #print(x.shape) batchSize,72,72
        h0 = torch.zeros(self.num_layers , x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers , x.size(0) , self.hidden_size).to(device)
        #print(h0.shape) 2,15,275
        #print(x.shape)
        
        #Forward Prop 
        
        out,_ =  self.lstm(x,(h0,c0))
        #print(out.shape)15 , 72,256
        out = out.reshape(out.shape[0] ,-1)
        #print(out.shape) 15,18432
        out = self.fc(out)
        #print(out.shape) 15 ,1214
        return out 

In [91]:
#load data 
train_loader = DataLoader(full_dataset , batch_size=batch_size , 
                          shuffle = True)
Validation_loader = DataLoader(validation_dataset , batch_size=batch_size , 
                          shuffle = False)
test_loader = DataLoader(Test_dataset  , batch_size=1 , shuffle=False)

In [92]:
model = RNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [93]:
#loss and optimizer 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= learning_rate)

In [94]:
# train Network
for epoch in range (num_epochs):
  for batch_ind ,(data, targets) in enumerate(train_loader):
    data = data.to(device = device).squeeze(1)
    targets = targets.to(device = device)
    # we are coping our data from the cpu to the GPU 

    #print(data.shape)

    # forward
    scores = model(data)
    # this were we send our data to with shape ([batchsize , chennel size , 36,36]in to
    # 15,1214 out as our predition )
    loss = criterion(scores , targets)
    # loss which is the mean square error of the scores which is the weights with the
    # targets which should be (15) and the yi-y

    # backword 

    optimizer.zero_grad()
    loss.backward()
    # compute the gradients 


    # gradient descent or adam step

    optimizer.step()
    #print(loss.item())

    



    #check the accuracy on validation data 

  def check_accuracy (loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
      for x,y in loader:
        x = x.to(device = device ).squeeze(1)
        y = y.to(device = device)
        # we are memicing the model and loading the data but this time with a 
        #trained model 
        
        scores = model(x)
        # our (15,1214) tensor 
        _,prediction = scores.max(1)
        # maximum value in the tensor (1) we need the index of it
        #print('model prediction = ',prediction , 'y = ',y)
        num_correct += (prediction == y).sum()
        # compare predition with y our target and get 0 for no and 1 for true or yes 
        num_samples += prediction.size(0)
        # compute the total num of samples 

      print(f'for{num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

  check_accuracy(train_loader,model)
  #check_accuracy(Validation_loader , model)


for12478/12906 with accuracy 96.68
for12687/12906 with accuracy 98.30
for12763/12906 with accuracy 98.89


In [96]:
test_loader = DataLoader(Test_dataset  , batch_size=1 , shuffle=False)
def write_csv_results (loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()
    your_file = open('resultsWithforeign_RNN_LSTM.csv', 'ab')
    # we create a csv file or open it 'ab' for append binery 
    sm = torch.nn.Softmax()
    # we use a softmax to compute the probability of each guess of the network 
    
    

    with torch.no_grad():
      for x,_ in loader:
        x = x.to(device = device ).squeeze(1)
       
        
        scores = model(x)
        _,prediction = scores.max(1)
        #print(scores.max())
        #print('model prediction = ',prediction , 'y = ',y)
        probabilities = sm(scores) 
        #print(probabilities.max())
        #print(prediction)
        if probabilities.max() < 0.25 :
          prediction = torch.tensor([-1])
          np.savetxt(your_file,prediction.cpu())
          # we save the prediction in the open file but first we have to 
          # send it back to being a cpu 
        else:
          np.savetxt(your_file,prediction.cpu().numpy())  
        
        

      model.train()
      your_file.close()

write_csv_results(test_loader,model)
# using our def 


C:\Users\yousi\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
